<img src = "https://github.com/VeryFatBoy/notebooks/blob/main/common/images/img_github_singlestore-jupyter_featured_2.png?raw=true">

<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/browser.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">Using SingleStore with OpenAI's Swarm</h1>
    </div>
</div>

In [7]:
!pip cache purge --quiet

In [8]:
!pip install git+https://github.com/openai/swarm.git --quiet
!pip install langchain --quiet
!pip install langchain-community --quiet
!pip install langchain-openai --quiet
!pip install pdf2image --quiet
!pip install pdfminer.six --quiet
!pip install unstructured==0.10.14 --quiet

In [12]:
import nltk
import shutil

from langchain.document_loaders import OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import SingleStoreDB
from langchain_openai import OpenAIEmbeddings
from singlestoredb.management import get_secret
from swarm import Swarm, Agent

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p>Select the database from the drop-down menu at the top of this notebook. It updates the <b>connection_url</b> which is used by SQLAlchemy to make connections to the selected database.</p>
    </div>
</div>

In [13]:
from sqlalchemy import *

conn = create_engine(connection_url)

In [14]:
nltk.download("punkt_tab")
nltk.download("averaged_perceptron_tagger_eng")

[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [15]:
loader = OnlinePDFLoader("https://www.investni.com/sites/default/files/2021-02/NI-fintech-document.pdf")

data = loader.load()

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [16]:
print (f"You have {len(data)} document(s) in your data")
print (f"There are {len(data[0].page_content)} characters in your document")

You have 1 document(s) in your data
There are 39674 characters in your document


In [17]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000,
    chunk_overlap = 20
)
texts = text_splitter.split_documents(data)

print (f"You have {len(texts)} pages")

You have 23 pages


In [18]:
os.environ["OPENAI_API_KEY"] = get_secret("OPENAI_API_KEY")

In [19]:
%%sql
DROP TABLE IF EXISTS fintech_docs;

++
||
++
++

In [20]:
embedding = OpenAIEmbeddings(model = "text-embedding-3-small")

docsearch = SingleStoreDB.from_documents(
    texts,
    embedding,
    table_name = "fintech_docs"
)

In [21]:
# Define the search function
def search_s2(query_text: str) -> str:
    # Perform a similarity search and return the first result's content
    docs = docsearch.similarity_search(query_text)
    return docs[0].page_content

# Define a function to transfer control to Agent B
def transfer_to_agent_b():
    return agent_b

In [23]:
# Define shared context to hold information between agents
shared_context = {}

# Define Agent A to focus on blockchain context
agent_a = Agent(
    name = "Blockchain Expert",
    model = "gpt-4o-mini",
    instructions = "You are an expert in blockchain technology.",
    functions = [search_s2, transfer_to_agent_b]
)

# Define Agent B to focus on investment opportunities
agent_b = Agent(
    name = "Investment Expert",
    model = "gpt-4o-mini",
    instructions = "You are an expert in stock market investment."
)

In [24]:
client = Swarm()

# Run Agent A with a blockchain-specific query
response_a = client.run(
    agent = agent_a,
    messages = [{
        "role": "user",
        "content": "What are the latest developments and trends in blockchain technology in Northern Ireland?"
    }],
)

# Store Agent A's response in shared context for Agent B to expand on
shared_context["blockchain_context"] = response_a.messages[-1]["content"]

# Output the response from Agent A
print(shared_context["blockchain_context"])

Northern Ireland is emerging as a significant hub for blockchain technology, particularly within its growing FinTech sector. Here are the latest developments and trends:

1. **FinTech as a Key Sector**: Northern Ireland has positioned itself as a center of excellence for financial technology (FinTech), leveraging its world-class talent, competitive operating costs, and a supportive business environment. This focus on FinTech has encouraged the integration of blockchain technology into various financial services.

2. **Corporate Investments**: Major international firms such as Citi, Allstate, Liberty Mutual, and FD Technologies are expanding their operations in Northern Ireland. Newer investors like CME Group, Vela, and TP ICAP have also established substantial delivery centers, indicating a growing confidence in the region's tech ecosystem.

3. **Start-Up Ecosystem**: The start-up and SME tech community in Northern Ireland is thriving. An entrepreneurial ecosystem bolstered by networki

In [25]:
# Run Agent B with a focus on investment opportunities within the blockchain context
response_b = client.run(
    agent = agent_b,
    messages = [{
        "role": "user",
        "content": f"{shared_context['blockchain_context']} Based on these developments, what are the best investment opportunities in blockchain in Northern Ireland?"
    }],
)

# Output the final response from Agent B
print(response_b.messages[-1]["content"])

Investing in blockchain technology in Northern Ireland, especially given its emerging status as a FinTech hub, presents several promising opportunities. Here are some potential investment avenues based on the latest developments and trends:

1. **Blockchain Startups**: With a thriving start-up ecosystem, consider investing in early-stage blockchain companies that are innovating within the FinTech space. Look for firms focusing on payment solutions, smart contracts, identity verification, and decentralization, as these can provide high growth potential.

2. **Corporate Ventures in Blockchain**: Given that major firms (Citi, Allstate, Liberty Mutual, etc.) are expanding their operations, exploring venture capital or equity investments in these corporations' blockchain initiatives or subsidiaries might yield returns as they evolve their platforms and products.

3. **Blockchain Infrastructure Companies**: Investment in companies developing the infrastructure necessary for blockchain implem

## Refactor Code

In [26]:
# Define the search function for Agent A
def search_s2(query_text: str) -> str:
    # Perform a similarity search and return the first result's content
    docs = docsearch.similarity_search(query_text)
    # Store the result in context for Agent B to access
    shared_context["blockchain_context"] = docs[0].page_content
    return docs[0].page_content

# Define a function to transfer control to Agent A
def transfer_to_agent_a():
    return agent_a

# Define a function to transfer control to Agent B
def transfer_to_agent_b():
    return agent_b

In [27]:
# Define shared context to hold information between agents
shared_context = {}

# Define Agent A to focus on blockchain context
agent_a = Agent(
    name = "Blockchain Expert",
    model = "gpt-4o-mini",
    instructions = "You are an expert in blockchain technology.",
    functions = [search_s2, transfer_to_agent_b]
)

# Define Agent B to focus on investment opportunities
agent_b = Agent(
    name = "Investment Expert",
    model = "gpt-4o-mini",
    instructions = "You are an expert in stock market investment.",
    functions = [transfer_to_agent_a]
)

In [28]:
client = Swarm()

# Run Agent A
response_a = client.run(
    agent = agent_a,
    messages = [{
        "role": "user",
        "content": (
            "What are the latest developments and trends in blockchain technology "
            "in Northern Ireland? Based on these developments, what are the best "
            "investment opportunities in blockchain in Northern Ireland?"
        )
    }],
    context_variables = {"blockchain_context": shared_context.get("blockchain_context", "")},
)

print(response_a.messages[-1]["content"])

### Latest Developments and Trends in Blockchain Technology in Northern Ireland

1. **FinTech Growth**: Northern Ireland is emerging as a center of excellence for FinTech, with significant investments from global firms like Citi, Allstate, and Liberty Mutual. This growth supports the development of blockchain technology within the financial services industry.

2. **Startup Ecosystem**: The startup and SME tech community in Northern Ireland is thriving, particularly in blockchain, due to a robust entrepreneurial ecosystem and a vibrant networking scene that promotes collaborative innovation.

3. **Technology Adoption**: Industries are increasingly looking to harness next-generation technologies, including blockchain, machine learning, and AI, to drive transformation. The growth in data and computing power, along with insights drawn from advanced analytics, is propelling this trend.

4. **Cybersecurity Focus**: With the rise of blockchain technology, there is a concurrent emphasis on cyb

## Cleanup

In [33]:
shutil.rmtree("nltk_data")

In [34]:
%%sql
DROP TABLE IF EXISTS fintech_docs;

++
||
++
++